# Pii Data Detection

Competition: **[The Learning Agency Lab - PII Data Detection](https://www.kaggle.com/competitions/pii-detection-removal-from-educational-data)**

- Develop automated techniques to detect and remove PII from educational data.

**SpaCy Token Alignment:**

![](spacy-roberta-tokens.png)

```
Text:         Design Thinking for innovation reflexion-Avril 2021-Nathalie Sylla
SpaCy Tokens: ['Design', 'Thinking', 'for', 'innovation', 'reflexion', '-', 'Avril', '2021', '-', 'Nathalie', 'Sylla']
SpaCy Labels: ['O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUDENT']

Model Tokens: ['[CLS]', '▁Design', '▁Thinking', '▁for', '▁innovation', '▁reflex', 'ion', '▁-', '▁Avril', '▁2021', '▁-', '▁Nathalie', '▁S', 'ylla']
Model Labels: ['-100', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'O', 'B-NAME_STUDENT', 'I-NAME_STUDENT', 'I-NAME_STUDENT']

SpaCy (a):    [(0, 'Design'), (1, 'Thinking'), (2, 'for'), (3, 'innovation'), (4, 'reflexion'), (5, '-'), (6, 'Avril'), (7, '2021'), (8, '-'), (9, 'Nathalie'), (10, 'Sylla')]
deBERTa (b):  [(0, '[CLS]'), (1, 'Design'), (2, 'Thinking'), (3, 'for'), (4, 'innovation'), (5, 'reflex'), (6, 'ion'), (7, '-'), (8, 'Avril'), (9, '2021'), (10, '-'), (11, 'Nathalie'), (12, 'S'), (13, 'ylla')]
a2b:          [[1], [2], [3], [4], [5, 6], [7], [8], [9], [10], [11], [12, 13]]
b2a:          [[], [0], [1], [2], [3], [4], [4], [5], [6], [7], [8], [9], [10], [10]]
```

**Change Logs**

- **[V2]** Added Inference Logic on output from previous model build.
- **[V1]** Basic PyTorch model working for training on  `epochs=3`, no inference.

**Notebooks**

- **[train]** [PII Data Detection - Deberta 1024](https://www.kaggle.com/code/gdataranger/pii-data-detection-deberta-1024)
- **[score]** [PII Data Detection - Deberta 1024 : SCORE](https://www.kaggle.com/code/gdataranger/pii-data-detection-deberta-1024-score)

**Requirements**

- Training
    - @gdataranger [TF2.16 requirements.txt](https://www.kaggle.com/datasets/gdataranger/tf2-16-requirements-txt)
    - HuggingFace [microsoft/deberta-v3-small](https://huggingface.co/microsoft/deberta-v3-small)

- Inference
    - @steubk [spacy_alignments-0.9.1](https://www.kaggle.com/datasets/steubk/spacy-alignments-0-9-1)
    - @gdataranger []()  need to create a model from training colab

- Production
    - HuggingFace [marksusol/deberta-v3-small-finetuned-ner-1024](https://huggingfa.co/marksusol/deberta-v3-small-finetuned-ner-1024)

**References**

1. HuggingFace
   - [token_classification-tf](https://colab.research.google.com/github/huggingface/notebooks/blob/main/examples/token_classification-tf.ipynb)
2. Spacy Alignment
    - [How to calculate the alignment between BERT and spaCy tokens effectively and robustly](https://gist.github.com/tamuhey/af6cbb44a703423556c32798e1e1b704)
3. Tensorflow
   - [tf.keras.metrics.FBetaScore](https://www.tensorflow.org/api_docs/python/tf/keras/metrics/FBetaScore)
   - [ValueError: (F1Score|FBetaScore) expects 2D inputs with shape (batch_size, output_dim). #771]()
4. Kaggle Contributors
    - @alejopaullier [PII | External Dataset](https://www.kaggle.com/datasets/alejopaullier/pii-external-dataset)
    - @valentinwerner [fix punctuation tokenization external dataset](https://www.kaggle.com/code/valentinwerner/fix-punctuation-tokenization-external-dataset)
    - @fhoughton [Train+Infer LLM | Deberta-v3](https://www.kaggle.com/code/fhoughton/train-infer-llm-deberta-v3/notebook)
    - @awsaf49 [PII Data Detection: KerasNLP Starter Notebook](https://www.kaggle.com/code/awsaf49/pii-data-detection-kerasnlp-starter-notebook/notebook)

## Setup

In [1]:
%%capture
# Following is based on https://github.com/huggingface/datasets/issues/6753#issuecomment-2027906687
if False:
    !pip install -U datasets==2.16.0
    !pip install -U fsspec==2023.10.0
    !pip install -U gcsfs==2023.10.0
    !pip install -U keras==3.0.5
    !pip install -U ml-dtypes==0.3.2
    !pip install -U numpy==1.26.4
    !pip install -U seqeval
    !pip install -U spacy-alignments==0.9.1
    !pip install -U tensorflow[with-cuda]~=2.16
    !pip install -U tensorflow-text~=2.16
    !pip install -U tf_keras~=2.16
    !pip install -U transformers==4.38.2
# Must restart session after this

In [2]:
import warnings

warnings.simplefilter(action='ignore', category=FutureWarning)
warnings.filterwarnings('ignore')

import logging
logging.basicConfig(level=logging.ERROR)

In [3]:
import os
os.environ["TOKENIZERS_PARALLELISM"] = 'true' # comes before import tensorflow
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'

import tensorflow as tf

assert tf.__version__ == '2.16.1'

print(f'TensorFlow: {tf.__version__}')

TensorFlow: 2.16.1


## Tokenization

In [4]:
# from ner import CFG, FILES, NerTagsDatasets, NerTagsTokenizer

"""Kaggle competition classes."""

import os
import json
import numpy as np
import pandas as pd
import random
import torch
import spacy_alignments as tokenizations
import transformers

from datasets import concatenate_datasets, Dataset, DatasetDict, ClassLabel, Sequence
from itertools import chain
from IPython.display import display, HTML
from tqdm import tqdm
from transformers import AutoTokenizer, AutoModelForTokenClassification

from google.colab import drive
drive.mount('/content/drive')

PATH_PREFIX = 'drive/MyDrive/Colab Notebooks/competition/'

FILES = {
    'train': [
         f'{PATH_PREFIX}/train.json',
         f'{PATH_PREFIX}/pii_dataset_fixed.json'],
    'test': [
         f'{PATH_PREFIX}/test.json']
}


class CFG:
    model_root = 'microsoft/deberta-v3-small'
    max_length = 1024
    model_checkpoint = f"{model_root.split('/')[1]}-finetuned-ner-{str(max_length)}"
    # Downloaded model files for local testing.
    #model_kaggle = f'{PATH_PREFIX}/input/deberta-v3-small-finetuned-ner-1024/transformers/1/1'
    hub_checkout = f'marksusol/{model_checkpoint}'

class NerTagsDatasets:

    def __init__(self, files, limit=False):
        self.id2tag = None
        self.ner_tag_mapping = None
        self.files = files
        self.limit = limit
        self.datasets = self.load_data(files)

    def load_data(self, files) -> Dataset:
        dataset = {}
        for split in files.keys():
            datasets = []
            for file in files[split]:
                ds = self.get_dataset(file)
                datasets.append(ds)
            dataset[split] = concatenate_datasets(datasets)

        # NerTags Mapping
        ner_tag_names = sorted(list(set(chain(*dataset['train']['ner_tags']))))
        # move '0' to Zero index, sort labels by A-B_C split.
        ner_tag_names = ner_tag_names[-1:] + sorted(ner_tag_names[:-1], key=lambda x: x.split('-')[1])
        ner_tag_mapping = {l: i for i, l in enumerate(ner_tag_names)}
        self.id2tag = {v: k for k, v in ner_tag_mapping.items()}
        ner_tags = [[ner_tag_mapping[label] for label in labels] for labels in dataset['train']['ner_tags']]

        train = dataset['train'].to_dict()
        train['ner_tags'] = ner_tags

        ds = DatasetDict({'train': Dataset.from_dict(train), 'test': dataset['test']})
        ds['train'] = ds['train'].cast_column('ner_tags', Sequence(
            feature=ClassLabel(num_classes=len(ner_tag_names), names=ner_tag_names)))
        return ds

    def get_dataset(self, filename) -> Dataset:
        data = json.load(open(filename))
        if self.limit:
            data = data[:500]
        dataset = {
            'full_text': [x['full_text'] for x in data],
            'document': [str(x['document']) for x in data],  # multiple train files w/ different type.
            'tokens': [x['tokens'] for x in data],
        }
        if 'labels' in data[0].keys():
            dataset['ner_tags'] = [x['labels'] for x in data]  # `label='B-NAME_STUDENT'` but need `ner_tags=4`
        return Dataset.from_dict(dataset)

    def show_random_elements(self, split='train', num_examples=10):
        assert num_examples <= len(self.datasets[split]), "Can't pick more elements than there are in the dataset."
        picks = []
        for _ in range(num_examples):
            pick = random.randint(0, len(self.datasets[split]) - 1)
            while pick in picks:
                pick = random.randint(0, len(self.datasets[split]) - 1)
            picks.append(pick)

        df = pd.DataFrame(self.datasets[split][picks])
        for column, typ in self.datasets[split].features.items():
            if isinstance(typ, ClassLabel):
                df[column] = df[column].transform(lambda i: typ.names[i])
            elif isinstance(typ, Sequence) and isinstance(typ.feature, ClassLabel):
                df[column] = df[column].transform(
                    lambda x: [typ.feature.names[i] for i in x]
                )
        display(HTML(df.to_html()))


class NerTagsTokenizer:

    def __init__(self, ds, checkpoint, is_train=True):
        self.tokenizer = None
        self.ner_tag_names = None
        self.num_labels = None
        self.label2id = None
        self.id2label = None
        self.dataset = ds
        self.checkpoint = checkpoint
        self.is_train = is_train
        self.set_tokenizer()
        if self.is_train:
            self.set_labels()
        self.tokenized_dataset = self.dataset.map(self.tokenize_and_align_labels, batched=True)
        if self.is_train:
            self.tokenized_datasets = self.get_split_tokenized_dataset()

    def set_labels(self):
        ner_tag_names = self.dataset.features['ner_tags'].feature.names
        self.label2id = {l: i for i, l in enumerate(ner_tag_names)}
        self.id2label = {v: k for k, v in self.label2id.items()}
        self.num_labels = len(ner_tag_names)
        self.ner_tag_names = ner_tag_names

    def set_tokenizer(self):
        self.tokenizer = AutoTokenizer.from_pretrained(self.checkpoint, add_prefix_space=True)
        assert isinstance(self.tokenizer, transformers.PreTrainedTokenizerFast)
        assert self.tokenizer.is_fast == True

    def tokenize_and_align_labels(self, inputs, label_all_tokens=True):
        tokenized_inputs = self.tokenizer(
            inputs['tokens'], is_split_into_words=True, truncation=True,
            padding='max_length', max_length=CFG.max_length,
            return_tensors='np'
        )
        if self.is_train:
            labels = []
            for i, ner_tags in enumerate(inputs['ner_tags']):
                word_ids = tokenized_inputs.word_ids(batch_index=i)
                previous_word_idx = None
                label_ids = []
                for word_idx in word_ids:
                    # Special tokens have a word id that is None.
                    # We set the label to -100, so they are automatically ignored in the loss function.
                    if word_idx is None:
                        label_ids.append(-100)
                    elif word_idx != previous_word_idx:
                        label_ids.append(ner_tags[word_idx])
                    else:
                        label_ids.append(ner_tags[word_idx] if label_all_tokens else -100)
                    previous_word_idx = word_idx

                labels.append(label_ids)
            tokenized_inputs['labels'] = labels
            tokenized_inputs['ner_tags'] = inputs['ner_tags']

        tokenized_inputs['document'] = inputs['document']
        tokenized_inputs['full_text'] = inputs['full_text']
        return tokenized_inputs

    def get_split_tokenized_dataset(self) -> Dataset:
        tokenized_datasets = self.tokenized_dataset.train_test_split(train_size=0.8, test_size=0.2, seed=42)
        tokenized_datasets['validation'] = tokenized_datasets['test']  # vs. train_validation_split
        tokenized_datasets.pop('test')
        return tokenized_datasets


class NerTagsPredict:

    def __init__(self, tokenized_ds, model_kaggle):
        self.tokenized_ds = tokenized_ds
        self.model = AutoModelForTokenClassification.from_pretrained(model_kaggle)
        self.tokenizer = AutoTokenizer.from_pretrained(model_kaggle)
        self.prediction_rows = self.get_prediction_rows()
        self.df = self.get_submission_df()

    def get_prediction_rows(self):
        prediction_rows = []
        for i, row in enumerate(tqdm(self.tokenized_ds)):
            document = row['document']
            # Alignment between DeBERTa and spaCy tokens in our input data.
            a = row['tokens']
            b = [self.tokenizer.decode(token) for token in row['input_ids']]
            a2b, b2a = tokenizations.get_alignments(a, b)

            input_ids = torch.tensor(row['input_ids']).unsqueeze(0)
            attention_mask = torch.tensor(row['attention_mask']).unsqueeze(0)

            logits = self.model(input_ids, attention_mask).logits
            predictions = torch.argmax(logits, dim=-1)
            prediction_labels = [self.model.config.id2label[i.item()] for i in predictions.squeeze()]

            for j, label in enumerate(prediction_labels):
                if label != 'O':
                    if len(b2a[j]) > 0:
                        row = [document, j, b2a[j][0], label]
                        prediction_rows.append(row)
        return prediction_rows

    def get_submission_df(self):
        sub = pd.DataFrame(self.prediction_rows, columns=['document', 'token', 'spacy', 'label'])
        sub['document'] = sub['document'].astype(int)
        sub = sub.groupby(['document', 'spacy'])['label'].agg(list).apply(np.unique).reset_index()
        sub.rename(columns={'spacy': 'token'}, inplace=True)

        sub = sub.sort_values(['document', 'token'], ascending=True)
        sub['label'] = sub['label'].apply(lambda x: x[0] if len(x) > 0 else 'O')
        return sub

    def to_csv(self, outfile='submission.csv'):
        self.df.to_csv(outfile, index_label='row_id')


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
'''
import json
import numpy as np

from datasets import Sequence, ClassLabel
from datasets import concatenate_datasets, Dataset, DatasetDict
from itertools import chain


import spacy_alignments as tokenizations
from transformers import AutoTokenizer

import json
import numpy as np

from datasets import concatenate_datasets, Dataset
from itertools import chain
from transformers import AutoTokenizer

import json
import numpy as np
import transformers

from datasets import concatenate_datasets, Dataset, DatasetDict
from itertools import chain

import numpy as np
import pandas as pd
import functools
from itertools import chain
from functools import partial
from pathlib import Path

import seqeval
from seqeval.metrics import recall_score, precision_score
from seqeval.metrics import classification_report
from seqeval.metrics import f1_score


import spacy_alignments as tokenizations
from transformers import AutoTokenizer

from datasets import ClassLabel, Sequence
import random
import pandas as pd
from IPython.display import display, HTML

# Transformer
from transformers import AutoTokenizer, Trainer, TrainingArguments
from transformers import AutoModelForTokenClassification, DataCollatorForTokenClassification
from datasets import Dataset, features
'''

'\nimport json\nimport numpy as np\n\nfrom datasets import Sequence, ClassLabel\nfrom datasets import concatenate_datasets, Dataset, DatasetDict\nfrom itertools import chain\n\n\nimport spacy_alignments as tokenizations\nfrom transformers import AutoTokenizer\n\nimport json\nimport numpy as np\n\nfrom datasets import concatenate_datasets, Dataset\nfrom itertools import chain\nfrom transformers import AutoTokenizer\n\nimport json\nimport numpy as np\nimport transformers\n\nfrom datasets import concatenate_datasets, Dataset, DatasetDict\nfrom itertools import chain\n\nimport numpy as np\nimport pandas as pd\nimport functools\nfrom itertools import chain\nfrom functools import partial\nfrom pathlib import Path\n\nimport seqeval\nfrom seqeval.metrics import recall_score, precision_score\nfrom seqeval.metrics import classification_report\nfrom seqeval.metrics import f1_score\n\n\nimport spacy_alignments as tokenizations\nfrom transformers import AutoTokenizer\n\nfrom datasets import ClassLa

In [6]:
ner_tags = NerTagsDatasets(FILES, limit=True)
ner_tags.datasets

Casting the dataset:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['full_text', 'document', 'tokens', 'ner_tags'],
        num_rows: 1000
    })
    test: Dataset({
        features: ['full_text', 'document', 'tokens'],
        num_rows: 10
    })
})

In [8]:
input = NerTagsTokenizer(ner_tags.datasets['train'], CFG.model_root)
tokenized_ds = input.tokenized_datasets
print(tokenized_ds)

Map:   0%|          | 0/1000 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['full_text', 'document', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 800
    })
    validation: Dataset({
        features: ['full_text', 'document', 'tokens', 'ner_tags', 'input_ids', 'token_type_ids', 'attention_mask', 'labels'],
        num_rows: 200
    })
})


## Training (Fine-Tuning)

Submissions are evaluated on micro **`F5`**, which is a classification metric that assigns value to recall and precision. The value of **`beta`**
 is set to 5, which means that **recall is weighted 5 times more heavily than precision**.

### TensorFlow Attempt

In [13]:
class FBetaScore(tf.keras.metrics.FBetaScore):
    def __init__(self, num_labels, ignore_classes=None, average="micro", beta=5.0, name="f5_score", **args):
        super().__init__(beta=beta, average=average, name=name, **args)
        self.num_labels = num_labels
        self.ignore_classes = ignore_classes or []

    def update_state(self, y_true, y_pred, sample_weight=None):
        y_true = tf.keras.ops.convert_to_tensor(y_true, dtype=self.dtype)
        y_pred = tf.keras.ops.convert_to_tensor(y_pred, dtype=self.dtype)

        y_true = tf.keras.ops.reshape(y_true, [-1])
        y_pred = tf.keras.ops.reshape(y_pred, [-1, self.num_labels])

        valid_mask = tf.keras.ops.ones_like(y_true, dtype=self.dtype)
        '''
        if self.ignore_classes:
            for ignore_class in self.ignore_classes:
                valid_mask &= tf.keras.ops.not_equal(y_true, tf.keras.ops.cast(ignore_class, y_pred.dtype))
        '''
        valid_mask = tf.keras.ops.expand_dims(valid_mask, axis=-1)

        y_true = tf.keras.ops.one_hot(y_true, self.num_labels)

        if not self._built:
            self._build(y_true.shape, y_pred.shape)

        threshold = tf.keras.ops.max(y_pred, axis=-1, keepdims=True)
        y_pred = tf.keras.ops.logical_and(
            y_pred >= threshold, tf.keras.ops.abs(y_pred) > 1e-9
        )

        y_pred = tf.keras.ops.cast(y_pred, dtype=self.dtype)
        y_true = tf.keras.ops.cast(y_true, dtype=self.dtype)

        tp = tf.keras.ops.sum(y_pred * y_true * valid_mask, self.axis)
        fp = tf.keras.ops.sum(y_pred * (1 - y_true) * valid_mask, self.axis)
        fn = tf.keras.ops.sum((1 - y_pred) * y_true * valid_mask, self.axis)

        self.true_positives.assign_add(tp)
        self.false_positives.assign_add(fp)
        self.false_negatives.assign_add(fn)

In [16]:
from transformers import (
    create_optimizer,
    DataCollatorForTokenClassification,
    TFAutoModelForTokenClassification
)

import keras

from transformers.keras_callbacks import KerasMetricCallback
from datasets import load_metric

class TFModelTrainer:
    def __init__(self, model, tokenizer, tokenized_ds):
        self.tf_train_set = None
        self.tf_validation_set = None
        self.model = model
        self.tokenizer = tokenizer
        self.batch_size = 4
        self.tokenized_ds = tokenized_ds
        self.tf_train_set, self.tf_validation_set = self.prepare_tf_datasets()
        self.optimizer, self.lr_schedule = self.get_optimizer()

    def prepare_tf_datasets(self):
        data_collator = DataCollatorForTokenClassification(
            tokenizer=self.tokenizer,
            return_tensors='np'
        )

        tf_train_set = model.prepare_tf_dataset(
            self.tokenized_ds['train'],
            shuffle=True,
            batch_size=self.batch_size,
            collate_fn=data_collator,
        )

        tf_validation_set = model.prepare_tf_dataset(
            self.tokenized_ds['validation'],
            shuffle=False,
            batch_size=self.batch_size,
            collate_fn=data_collator,
        )
        return tf_train_set, tf_validation_set

    def get_optimizer(self):
        num_train_epochs = 1 # TODO: MAKE 10
        num_train_steps = (len(self.tokenized_ds['train']) // self.batch_size) * num_train_epochs
        optimizer, lr_schedule = create_optimizer(
            init_lr=2e-5,
            num_train_steps=num_train_steps,
            weight_decay_rate=0.01,
            num_warmup_steps=0,
        )
        return optimizer, lr_schedule

    # Compute the model performance metrics
    def get_fbeta_score(self, precision, recall, beta=5.0):
        b2 = beta ** 2
        return (1 + b2) * ((precision * recall) / (b2 * precision + recall))

    def compute_metrics(self, eval_preds):
        logits, labels = eval_preds
        predictions = np.argmax(logits, axis=-1)
        true_labels = [[input.ner_tag_names[l] for l in label if l!=-100] for label in labels]

        true_predictions = [[input.ner_tag_names[p] for p,l in zip(prediction, label) if l!=-100]
                            for prediction, label in zip(predictions, labels)]

        metric = load_metric('seqeval')
        results = metric.compute(predictions=true_predictions, references=true_labels)

        # Competition requirement.
        precision = results['overall_precision']
        recall = results['overall_recall']
        f_beta_score = self.get_fbeta_score(precision, recall, beta=5.0)

        return {"precision": results['overall_precision'],
                "recall": results['overall_recall'],
                "fbeta": f_beta_score,  # 'f1': results['overall_f1']
                "accuracy": results['overall_accuracy']}


    def fit(self, num_train_epochs=1):
        # ValueError: FBetaScore expects 2D inputs with shape (batch_size, output_dim). Received input shapes: y_pred.shape=(4, 1024, 12) and y_true.shape=(4, 1024).
        #fbeta_score = keras.metrics.FBetaScore(average='micro', beta=5.0, threshold=0.6, name='fbeta_score')
        fbeta_score = tf.keras.metrics.FBetaScore(average='micro', beta=5.0, threshold=0.6, name='fbeta_score')
        self.model.compile(optimizer=self.optimizer, metrics=[fbeta_score])


        ''' https://www.kaggle.com/code/awsaf49/pii-data-detection-kerasnlp-starter-notebook/comments#2741028
        File "<ipython-input-13-beb90441fef0>", line 22, in update_state
        y_true = tf.keras.ops.one_hot(y_true, self.num_labels)
        File "/usr/local/lib/python3.10/dist-packages/keras/src/ops/nn.py", line 1305, in one_hot
            return backend.nn.one_hot(
        File "/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py", line 449, in one_hot
            return tf.one_hot(x, num_classes, axis=axis, dtype=dtype)

        TypeError: Value passed to parameter 'indices' has DataType float32 not in list of allowed values: uint8, int8, int32, int64

        self.model.compile(optimizer=self.optimizer, metrics=[FBetaScore(num_labels=input.num_labels)])
        '''

        # works - self.model.compile(optimizer=self.optimizer) # No loss argument!
        metric_callback = KerasMetricCallback(metric_fn=self.compute_metrics, eval_dataset=self.tf_validation_set)
        self.model.fit(x=self.tf_train_set, validation_data=self.tf_validation_set, epochs=num_train_epochs, callbacks=[metric_callback])

model = TFAutoModelForTokenClassification.from_pretrained(
    CFG.model_root,
    num_labels=input.num_labels,
    id2label=input.id2label,
    label2id=input.label2id
)

trainer = TFModelTrainer(model, input.tokenizer, tokenized_ds)
with tf.device('/device:GPU: 0'):
    trainer.fit()

All model checkpoint layers were used when initializing TFDebertaV2ForTokenClassification.

Some layers of TFDebertaV2ForTokenClassification were not initialized from the model checkpoint at microsoft/deberta-v3-small and are newly initialized: ['classifier', 'dropout_6']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


ValueError: in user code:

    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py", line 1398, in train_function  *
        return step_function(self, iterator)
    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/training.py", line 1370, in run_step  *
        outputs = model.train_step(data)
    File "/usr/local/lib/python3.10/dist-packages/transformers/modeling_tf_utils.py", line 1643, in train_step  *
        self.compiled_metrics.update_state(y, y_pred, sample_weight)
    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/engine/compile_utils.py", line 620, in update_state  *
        metric_obj.update_state(y_t, y_p, sample_weight=mask)
    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/metrics/base_metric.py", line 153, in decorated  *
        result = update_state_fn(*args, **kwargs)
    File "/usr/local/lib/python3.10/dist-packages/tf_keras/src/metrics/base_metric.py", line 140, in update_state_fn  *
        return